In [ ]:
%pip install --quiet langchain langchain_community langgraph langchain_openai langgraph-supervisor tavily-python httpx

In [9]:
# Set API Keys
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-your-open-ai-key"
os.environ["TAVILY_API_KEY"] = "tvly-dev-key"

In [2]:
from typing import Dict, Any
import httpx
import json

from tavily import TavilyClient
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"], model="gpt-4o-mini", temperature=0.1)
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])


In [ ]:
@tool
def extract_documentation_from_website(url: str) -> str:
    """
    Extract documentation content from a website URL.
    This tool helps the agent to extract documentation content from a website URL and learn about the API endpoints and parameters from the website.
    """
    try:
        content_response = tavily_client.extract(urls=url)

        if content_response and len(content_response) > 0:
            return f"Documentation extracted from {url}:\n\n{content_response}"
       
        else:
            return f"Could not extract content from {url}. Please check the URL and try again."
            
    except Exception as e:
        return f"Error extracting documentation from {url}: {str(e)}"

In [ ]:
@tool
def create_documentation_summary(content: str) -> str:
    """
    Create a summary of the documentation content.
    This tool helps the agent to create a summary of the documentation content to learn about the API endpoints and parameters from the website.
    It cleans the content to remove all the noise and keep only the most important information.
    Focus on the API endpoints and parameters usage.
    """

    system_prompt_summary_agent = """
    <CONTEXT>
    You are an expert in the creation of API documentation summaries.
    You are given a raw documentation content and you need to create a summary of the documentation that will be used by another agent to learn about the API endpoints and parameters 
    that are necessary to make API calls.
    </CONTEXT>
    
    Be very specific and organize the content with the following JSON format:
    
    <OUTPUT_FORMAT>
    "endpoints": [
        {{
            "name": "endpoint_name",
            "description": "endpoint_description",
            "parameters": [
                {{
                    "name": "parameter_name",
                    "description": "parameter_description",
                    "required": "yes" or "no"
                }}
            ]
        }}
    </OUTPUT_FORMAT>
    Do not lose any information and be very specific, your summary must be based only on the content provided.
    """
    
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt_summary_agent),
        ("user", "Here is the content to be summarized: {content}")
    ])

    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"content": content})
    
    return response

In [ ]:

# List of tools for the agent
tools = [extract_documentation_from_website, create_documentation_summary]

In [ ]:
@tool
def fetch_api_data(endpoint: str, parameters: Dict[str, Any] = None) -> str:
    """
    Fetch data from an API endpoint using the learned parameters.
    This tool performs HTTP requests to API endpoints after the agent
    has learned how to use them from documentation.
    
    Args:
        endpoint (str): The API endpoint URL to fetch data from
        parameters (Dict[str, Any]): Query parameters to include in the request
        
    Returns:
        str: The API response data as a formatted string
    """
    try:
        # Build the URL with parameters
        if parameters:
            # Convert parameters to query string
            param_strings = []
            for key, value in parameters.items():
                param_strings.append(f"{key}={value}")
            if param_strings:
                endpoint += "?" + "&".join(param_strings)
        
        # Make the HTTP request
        with httpx.Client() as client:
            response = client.get(endpoint, timeout=30.0)
            response.raise_for_status()
            
            # Try to parse as JSON, fallback to text
            try:
                data = response.json()
                return f"API Response from {endpoint}:\n\n{json.dumps(data, indent=2)}"
            except json.JSONDecodeError:
                return f"API Response from {endpoint}:\n\n{response.text}"
                
    except httpx.HTTPStatusError as e:
        return f"HTTP Error {e.response.status_code} when fetching {endpoint}: {e.response.text}"
    except httpx.TimeoutException:
        return f"Timeout error when fetching {endpoint}. The request took too long."
    except Exception as e:
        return f"Error fetching data from {endpoint}: {str(e)}"


In [ ]:
system_prompt = """
You are a specialized API assistant that helps users learn and interact with APIs. 
    
You have access to the following tools:
- extract_documentation_from_website(url: str) -> str:
    - url: The URL of the documentation website to extract content from
    Call for this tool to extract all the content from the documentation website

- create_documentation_summary(content: str) -> str:
    - content: The content to be summarized
    Call for this tool to create a summary of the documentation content extracted from the documentation website    

We create a summary of the documentation content extracted from the documentation website to be used by another agent to learn about the API endpoints and parameters from the website
When we extract the content from the documentation website it returns a lot of information, but we only need the information about the API endpoints and parameters.
Thats why we need to create a summary of the documentation content extracted from the documentation website eliminating all the noise and keeping only useful information.

You should:
- Analyze if you already have the summary of the documentation content extracted from the documentation website
- If you don't have the summary of the documentation content extracted from the documentation website, 
- call for the extract_documentation_from_website tool to extract the content from the documentation website
- call for the create_documentation_summary tool to create a summary of the documentation content extracted from the documentation website

- Analyze the summary of the documentation content to understand available endpoints and parameters
- Returns the summary of the documentation content extracted from the documentation website
"""

In [ ]:
# Create the LangGraph react agent
agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt=system_prompt
)

In [7]:
inputs = {"messages": [{"role": "user", "content": "Tell me about all available endpoints and parameters of the OpenF1 API https://openf1.org/"}]}
for chunk in agent.stream(inputs, stream_mode="updates"):
    print(chunk)


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2Fstt9K7yJOHjAiT18B85UBp', 'function': {'arguments': '{"url":"https://openf1.org/"}', 'name': 'extract_documentation_from_website'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 479, 'total_tokens': 503, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CGGFxK0N5mMJndeyz7gS782MT0zOG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6ea47fdc-0026-479e-b87b-27e864872950-0', tool_calls=[{'name': 'extract_documentation_from_website', 'args': {'url': 'https://openf1.org/'}, 'id': 'call_2Fstt9K7yJOHjAiT18B85UBp', 'type': 'tool_call'}], usage_meta